In [1]:
import tensorflow as tf
import keras
#import matplotlib.pyplot as plt
#%matplotlib inline
import numpy as np
import json
import pandas as pd
from statistics import mean
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam #SGD, Adadelta, RMSprop
#from sklearn import preprocessing

In [10]:
config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 4} ) 
sess = tf.compat.v1.Session(config=config) 
keras.backend.set_session(sess)

In [2]:
# Open and Read JSON file's data

file = open(r"shallow-frequencies/shallow-frequencies.json", "r", encoding='utf-8')
js = file.read()
data = json.loads(js)
file.close()

In [3]:
# Get all the proofs
proofs = data["triples"]

types = [row["name"] for row in data["types"]]
terms = [row["name"] for row in data["terms"]]

In [4]:
input_list = []
output_list = []

for proof in proofs:
    input_def = [0]*len(types)
    output_def = [0]*len(terms)
    
    for typ in proof['types']:
        ind = types.index(typ)
        input_def[ind] +=1
        
    input_list.append(input_def)
    
    for trm in proof['terms']:
        ind = terms.index(trm)
        output_def[ind] +=1
        
    output_list.append(output_def)

In [5]:
input_list = np.asarray(input_list)
output_list = np.asarray(output_list)

In [6]:
#no_test = 1000 no_test/len(proofs)
x_train, x_test, y_train, y_test = train_test_split(input_list, output_list, test_size=0.2)

In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  

In [10]:
model = Sequential()
model.add(Dense(200, input_shape = (709,), activation = "softplus"))
model.add(Dropout(0.2))
model.add(Dense(3337, activation = "softplus"))
model.compile(Adam(learning_rate = 0.01), "poisson", metrics = ["KLDivergence"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 200)               142000    
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 3337)              670737    
                                                                 
Total params: 812,737
Trainable params: 812,737
Non-trainable params: 0
_________________________________________________________________


In [27]:
model = Sequential()
model.add(Dense(200, input_shape = (709,), activation = "softplus"))
model.add(Dropout(0.2))
model.add(Dense(3337, activation = "softplus"))
model.compile(Adam(learning_rate = 0.01), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 200)               142000    
                                                                 
 dropout_4 (Dropout)         (None, 200)               0         
                                                                 
 dense_9 (Dense)             (None, 3337)              670737    
                                                                 
Total params: 812,737
Trainable params: 812,737
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train, epochs = 30)

In [ ]:
model.fit(x_train, y_train, epochs = 20, validation_data=(x_test, y_test))

In [44]:
#model.save('Model')

In [45]:
#model = keras.models.load_model('Model')

In [12]:
model.evaluate(x_test, y_test)

43/43 [==============================] - 0s 7ms/step - loss: 0.0067 - kullback_leibler_divergence: 15.1435


[0.006681405007839203, 15.143548011779785]

In [13]:
def get_score(y_test, y_pred):
    
    df = pd.DataFrame(list(zip(y_test, y_pred)), columns =['y_test', 'y_pred'])
    df = df.sort_values(by="y_pred", ascending=False)
    
    df['y_pred'] = range(len(df))
    sum_val = df['y_test'].sum()
    
    if sum_val:
        return (df['y_test']*df['y_pred']).sum()/sum_val
    else:
        return 0

In [14]:
def avg_score(x_tests, y_tests):
    
    y_preds = model.predict(x_tests)
    print('\n',f"Calculating average score for {len(y_preds)} elements.")
    
    list_score = []
    
    for i in range(len(y_preds)):
        score = get_score(y_tests[i], y_preds[i])
        list_score.append(score)
    
    avg_per = mean(list_score)
    print(f" Average rank elements is {round(avg_per)} out of {len(data['terms'])} total terms.")
    
    return avg_per

In [15]:
avg_score(x_test, y_test)

43/43 [==============================] - 0s 2ms/step

 Calculating average score for 1365 elements.
 Average rank elements is 294 out of 3337 total terms.


294.17679624219056

In [15]:
y_pred = model.predict(x_test)
cut_off = 10

for i in range(len(y_test)):
    df = pd.DataFrame(list(zip(terms, y_test[i], y_pred[i])), columns =['terms', 'y_test_count', 'y_pred_index'])
    df = df.sort_values(by="y_pred_index", ascending=False)

    df['y_pred_index'] = range(len(df))
    sum_val = df['y_test_count'].sum()

    df.index = range(len(df))

    if sum_val:
        avg = (df['y_test_count']*df['y_pred_index']).sum()/(sum_val)
    else:
        avg = 0
        
    if avg <= cut_off:
        print('\n -  -  -  \n')
        if str(avg) == "0.0":
            print('Average', avg)
            display(df[:1])
        elif avg > 0:
            print('Average', avg)
            display(df[:df['y_test_count'].iloc[::-1].ne(0).idxmax()+1])
        else:
            print('Average', avg, '(no elements in defn)')

7/7 [==============================] - 0s 3ms/step

 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 0.5


,terms,y_test_count,y_pred_index
0,Nat,3,0
1,OfNat.ofNat,3,1



 -  -  -  

Average 0 (no elements in defn)

 -  -  -  

Average 4.0


,terms,y_test_count,y_pred_index
0,Nat,0,0
1,Eq,0,1
2,Eq.refl,0,2
3,Eq.symm,0,3
4,rfl,1,4



 -  -  -  

Average 6.428571428571429


...


 -  -  -  

Average 5.827586206896552


,terms,y_test_count,y_pred_index
0,Int,14,0
1,Int.ofNat,0,1
2,HAdd.hAdd,1,2
3,Neg.neg,1,3
4,Nat,0,4
5,Eq,2,5
6,Int.instNegInt,1,6
7,OfNat.ofNat,2,7
8,rfl,1,8
9,Int.add_left_comm,0,9



 -  -  -  

Average 7.5


,terms,y_test_count,y_pred_index
0,UInt16.val,2,0
1,UInt16,0,1
2,UInt16.mk,1,2
3,OfNat.ofNat,0,3
4,Nat,0,4
5,UInt16.modn,0,5
6,UInt16.size,0,6
7,Nat.toUInt16,0,7
8,Fin.mk,0,8
9,HMod.hMod,0,9



 -  -  -  

Average 0 (no elements in defn)
